In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/suriya/dcu/Group-Project


In [21]:
from src.ndcf import score_me, validate_me
import pandas as pd

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Make a prediction

In [34]:
from src.featutils import aggregate_features_from_folder
from src.features.rssi_distance import extract_feature2, postproc_feature_dicts

In [73]:
dev_dir = "data/tc4tl_data_v5/tc4tl/data/dev/"
test_dir = "data/tc4tl_data_v5/tc4tl/data/test/"
dev_key = pd.read_csv("data/tc4tl_data_v5/tc4tl/docs/tc4tl_dev_key.tsv", sep="\t")
test_key = pd.read_csv("data/tc4tl_test_key/tc4tl/docs/tc4tl_test_key.tsv", sep="\t")

In [82]:
devset, _ = aggregate_features_from_folder(dev_dir, dev_key, extract_feature2, postproc_feature_dicts, verbose=True,
                                          tunables={'N' : 0.3315, 'TX' : -63.982}, testing=False)
testset, _ = aggregate_features_from_folder(test_dir, test_key, extract_feature2, postproc_feature_dicts, verbose=True,
                                          tunables={'N' : 0.3315, 'TX' : -63.982}, testing=False)

100%|██████████| 8423/8423 [00:06<00:00, 1299.99it/s]


### Evaluation on Dev Set

In [83]:
from sklearn.ensemble import RandomForestClassifier

devset['CoarseGrain'] = devset['CoarseGrain'].replace({'Y' : 0, 'N' : 1})
X = devset[['PredictedDistance', 'CoarseGrain']]
y = devset.Distance
rf = RandomForestClassifier()
rf.fit(X, y)


RandomForestClassifier()

In [84]:
ypred = rf.predict(X)
sysout = pd.DataFrame({
        'fileid': devset['fileid'],
        'distance': ypred
    })

In [85]:
from src.ndcf import compute_ndcf

compute_ndcf(sysout)

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.006536,0.0,0.006536
1,fine_grain,1.8,0.000000,0.0,0.000000
2,fine_grain,3.0,0.000000,0.0,0.000000
3,coarse_grain,1.8,0.000000,0.0,0.000000


### Evaluation on test set

In [86]:
testset['CoarseGrain'] = testset['CoarseGrain'].replace({'Y' : 0, 'N' : 1})
X = testset[['PredictedDistance', 'CoarseGrain']]
y = testset.Distance
rf = RandomForestClassifier()
rf.fit(X, y)
ypred = rf.predict(X)
sysout = pd.DataFrame({
        'fileid': testset['fileid'],
        'distance': ypred
    })

In [87]:
compute_ndcf(sysout)

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.001421,0.001908,0.003330
1,fine_grain,1.8,0.000355,0.002514,0.002870
2,fine_grain,3.0,0.000238,0.001430,0.001668
3,coarse_grain,1.8,0.001405,0.000000,0.001405
